In [12]:
import pandas as pd
import os
from pathlib import Path
from pathlib import Path
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline

In [2]:
file = Path('../../../data/cleandata/CMC_df.csv')
tickers_df = pd.read_csv(file)
tickers_df.head()

,rank_cnc,Name,ticker,market_cap,price,volume24,pct1h,pct24h,pct7d
0,1,Bitcoin,BTC,"156,629,905,950","8,678.58","19,216,389,896",-0.28,-1.35,-6.14
1,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96
2,3,XRP,XRP,"11,612,597,542",0.268199,"1,265,885,288",-0.57,-1.75,-7.36
3,4,Bitcoin Cash,BCH,"5,016,918,192",276.97,"1,842,963,894",-0.75,-3.01,-5.39
4,5,Tether,USDT,"4,135,369,515",1.01,"22,054,927,684",-0.08,0.36,0.34


In [3]:
tickers_dict = tickers_df.set_index('ticker').to_dict()['Name']

In [4]:
file2 = Path('./tickers.csv')
tickers_df = pd.read_csv(file2)
tickers_df.dropna(inplace=True)
tickers_df.sort_values(by=['End'])
tickers_df = tickers_df.iloc[0:720]
tickers_dict2 = tickers_df.set_index('Ticker').to_dict()['Name']

In [5]:
len(tickers_dict)

2270

In [6]:
len(tickers_dict2)

703

In [7]:
new_dict = {}
for key, value in tickers_dict.items():
    if key not in tickers_dict2:
        new_dict[key] = value

In [8]:
len(new_dict)

1954

In [15]:
dict_group1 = {k: new_dict[k] for k in sorted(new_dict.keys())[:1000]}

In [40]:
dict_group2 = {k: new_dict[k] for k in sorted(dict_group1.keys())[247:1000]}

In [41]:
len(dict_group2)

753

In [13]:
import webhoseio
api_key = os.getenv("WEBHOSE_API_2")
webhoseio.config(token=api_key)

In [44]:
articles_dict = {}
for key, value in dict_group2.items():
    try:
        articles_dict[key] = webhoseio.query("filterWebContent", {"q": f"crypto AND ({key} OR {value})", "language":"english"})
        
    except:
        pass

In [45]:
len(articles_dict)

672

In [46]:
sentiment_df = pd.DataFrame()
analyzer = SentimentIntensityAnalyzer()

In [47]:
for key in articles_dict:
    sentiments = []
    for article in articles_dict[key]["posts"]:
        try:
            sentiment = analyzer.polarity_scores(article["text"])
      
            sentiments.append({
                "ticker": key,
                "compound": sentiment["compound"],
                "positive": sentiment["pos"],
                "negative": sentiment["neg"],
                "neutral": sentiment["neu"]
            
            })
        
        except AttributeError:
            pass
      
    sentiment_df = sentiment_df.append(pd.DataFrame(sentiments))

In [48]:
sentiment_df.head()

,ticker,compound,positive,negative,neutral
0,BPRO,0.9929,0.169,0.025,0.806
1,BPRO,0.9954,0.080,0.012,0.908
2,BPRO,0.8239,0.070,0.058,0.872
3,BPRO,0.9989,0.104,0.025,0.871
4,BPRO,0.9981,0.035,0.015,0.950


In [49]:
len(sentiment_df)

23472

In [50]:
sentiment_df = sentiment_df.set_index("ticker")

In [51]:
sentiment_avg = sentiment_df.groupby(['ticker']).mean()
sentiment_avg.head()

,compound,positive,negative,neutral
ticker,,,,
BPRO,0.613429,0.095810,0.040670,0.863480
BPT,0.085700,0.013000,0.014000,0.973000
BQT,-0.999700,0.059000,0.080000,0.860000
BQTX,-0.177867,0.006333,0.011333,0.982333
BRAT,0.220290,0.094581,0.086720,0.818731


In [52]:
sentiment_count = sentiment_df.groupby(['ticker']).count()
sentiment_count.head()

,compound,positive,negative,neutral
ticker,,,,
BPRO,100,100,100,100
BPT,1,1,1,1
BQT,1,1,1,1
BQTX,3,3,3,3
BRAT,93,93,93,93


In [53]:
sentiment_count = sentiment_count.drop(columns=['negative','neutral','positive'])
sentiment_count = sentiment_count.rename(columns={'compound':'no_of_posts'})
sentiment_count.head()

,no_of_posts
ticker,
BPRO,100
BPT,1
BQT,1
BQTX,3
BRAT,93


In [54]:
ticker_sentiments = pd.concat([sentiment_avg, sentiment_count], axis=1)
ticker_sentiments.head()

,compound,positive,negative,neutral,no_of_posts
ticker,,,,,
BPRO,0.613429,0.095810,0.040670,0.863480,100
BPT,0.085700,0.013000,0.014000,0.973000,1
BQT,-0.999700,0.059000,0.080000,0.860000,1
BQTX,-0.177867,0.006333,0.011333,0.982333,3
BRAT,0.220290,0.094581,0.086720,0.818731,93


In [55]:
file = Path(f"./ticker_sentiments.csv")
ticker_sentiments.to_csv(file, mode='a', header=False)